In [2]:
import os

In [3]:
%pwd

'd:\\Mlops\\Chicken-Disease-Classification-Project\\research'

In [4]:
os.chdir("d:/Mlops/Chicken-Disease-Classification-Project/")

In [5]:
%pwd

'd:\\Mlops\\Chicken-Disease-Classification-Project'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from chicken_disease_classification.utils.common import read_yaml_file, create_directories
from chicken_disease_classification.constants import *

In [8]:
import chicken_disease_classification.utils.common as common
print(dir(common)) 

['Any', 'BoxValueError', 'Path', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'base64', 'config_box', 'create_directories', 'decode_image', 'encode_image', 'get_size', 'joblib', 'json', 'load_bin', 'logger', 'os', 'read_yaml_file', 'save_bin', 'save_json', 'yaml']


In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath: Path = CONFIG_FILE_PATH,
        params_filepath: Path = PARAMS_FILE_PATH,
    ):
        self.config = read_yaml_file(config_filepath)
        self.params = read_yaml_file(params_filepath)
        create_directories([Path(self.config.artifacts_root)])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir),
        )

        return data_ingestion_config

In [10]:
import os
import urllib.request as request
import zipfile
from chicken_disease_classification import logger
from chicken_disease_classification.utils.common import get_size


In [15]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_data(self) -> Path:
        logger.info("Starting data download...")
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file,
            )
            logger.info(
                f"File: {filename} downloaded with info: {headers} and size: {get_size(Path(filename))}"
            )
        else:
            logger.info(f"File already exists of size: {get_size(self.config.local_data_file)}")

        return self.config.local_data_file

    def extract_zip_file(self, zip_file_path: Path):
        logger.info("Starting data extraction...")
        with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
            zip_ref.extractall(self.config.unzip_dir)
        logger.info(f"Data extracted at location: {self.config.unzip_dir}")

In [14]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    
    # Create required directories using your utility function
    create_directories([
        Path(data_ingestion_config.local_data_file).parent,
        Path(data_ingestion_config.unzip_dir)
    ])
    
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_zip_file(zip_file_path=data_ingestion_config.local_data_file)
except Exception as e:
    raise e

2025-11-30 12:43:14,982 - chicken_disease_classification - INFO - YAML file: config\config.yaml loaded successfully
2025-11-30 12:43:14,985 - chicken_disease_classification - INFO - YAML file: params.yaml loaded successfully
2025-11-30 12:43:14,988 - chicken_disease_classification - INFO - Directory created at: artifacts
2025-11-30 12:43:14,991 - chicken_disease_classification - INFO - Directory created at: artifacts\data_ingestion
2025-11-30 12:43:14,992 - chicken_disease_classification - INFO - Directory created at: artifacts\data_ingestion
2025-11-30 12:43:14,993 - chicken_disease_classification - INFO - Starting data download...
2025-11-30 12:43:28,246 - chicken_disease_classification - INFO - File: artifacts\data_ingestion\data.zip downloaded with info: Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d86